In [1]:
import optuna
from uuid import uuid1
from functools import partial
import ConfigSpace as CS
from hpobench.util.openml_data_manager import get_openml100_taskids

In [2]:
def sample_config_from_optuna(trial, cs):
    # https://github.com/automl/HPOBenchExperimentUtils/blob/9ed90230880970dd93bbc4ec51813ffd37e2654b/HPOBenchExperimentUtils/optimizer/optuna_optimizer.py
    config = {}
    for hp_name in cs:
        hp = cs.get_hyperparameter(hp_name)
        if isinstance(hp, CS.UniformFloatHyperparameter):
            value = float(trial.suggest_float(name=hp_name, low=hp.lower, high=hp.upper, log=hp.log))
        elif isinstance(hp, CS.UniformIntegerHyperparameter):
            value = int(trial.suggest_int(name=hp_name, low=hp.lower, high=hp.upper, log=hp.log))
        elif isinstance(hp, CS.CategoricalHyperparameter):
            hp_type = type(hp.default_value)
            value = hp_type(trial.suggest_categorical(name=hp_name, choices=hp.choices))
        elif isinstance(hp, CS.OrdinalHyperparameter):
            num_vars = len(hp.sequence)
            index = trial.suggest_int(hp_name, low=0, high=num_vars - 1, log=False)
            hp_type = type(hp.default_value)
            value = hp.sequence[index]
            value = hp_type(value)
        else:
            raise ValueError(f'Please implement the support for hps of type {type(hp)}')

        config[hp.name] = value
    return config

In [3]:
def get_budget_and_fidelity_type(main_fidelity):
    if isinstance(main_fidelity, CS.OrdinalHyperparameter):
        print("A")
        order = main_fidelity.get_seq_order()
        min_budget = main_fidelity.get_value(order.min())
        max_budget = main_fidelity.get_value(order.max())
    else:
        print("B")
        min_budget = main_fidelity.lower
        max_budget = main_fidelity.upper

    if isinstance(main_fidelity, CS.UniformIntegerHyperparameter):
        # TODO: We need to fix this through never setting an int budget to 0
        min_budget = max(min_budget, 1)
        max_budget = max(max_budget, 1)
    else:
        print(min_budget)
        min_budget = max(min_budget, 0.01)
        max_budget = max(max_budget, 0.01)

    if isinstance(main_fidelity, CS.hyperparameters.FloatHyperparameter):
        main_fidelity_type = float
    elif isinstance(main_fidelity, CS.hyperparameters.IntegerHyperparameter):
        main_fidelity_type = int
    elif isinstance(main_fidelity, CS.CategoricalHyperparameter):
        main_fidelity_type = type(main_fidelity.default_value)
    else:
        main_fidelity_type = type(main_fidelity.default_value)
    return main_fidelity_type, min_budget, max_budget

In [4]:
def objective_fn(trial, benchmark, main_fidelity_name, main_fidelity_type, max_budget, configspace):
    configuration = sample_config_from_optuna(trial, configspace)
    result_dict = benchmark.objective_function(
        configuration, {main_fidelity_name: main_fidelity_type(max_budget)}
    )
    trial.report(result_dict['function_value'], step=max_budget)
    return result_dict['function_value']

In [5]:
def run_benchmark(sampler, benchmark, main_fidelity, n_trials=20):
    study = optuna.create_study(direction='minimize', sampler=sampler)

    cs = benchmark.get_configuration_space()
    main_fidelity_type, min_budget, max_budget = get_budget_and_fidelity_type(main_fidelity)

    # noinspection PyTypeChecker
    study.optimize(func=partial(objective_fn,
                                benchmark=benchmark,
                                main_fidelity_name=main_fidelity.name,
                                main_fidelity_type=main_fidelity_type,
                                max_budget=max_budget,
                                configspace=cs),
                   timeout=None, n_trials=n_trials)  # Run the optimization without a limitation
    return study

In [16]:
from ghosttuner.samplers.llm_sampler import GhosttunerSampler
from optuna.samplers import RandomSampler, TPESampler, CmaEsSampler
from hpobench.benchmarks.ml.nn_benchmark import NNBenchmark

openml100_task_ids = get_openml100_taskids()
benchmark = NNBenchmark(openml100_task_ids[0])

In [17]:
benchmark

In [18]:
benchmark.train_y

1163    5
5184    5
1872    8
4628    9
4859    7
       ..
417     1
2783    6
1701    3
1162    6
4140    7
Name: class, Length: 2523, dtype: category
Categories (10, object): ['0' < '1' < '2' < '3' ... '6' < '7' < '8' < '9']

In [19]:
search_space = "\n".join([str(hp) for hp in benchmark.get_configuration_space().get_hyperparameters()])
print(search_space)

alpha, Type: UniformFloat, Range: [1e-08, 1.0], Default: 0.001, on log-scale
batch_size, Type: UniformInteger, Range: [4, 256], Default: 32, on log-scale
depth, Type: UniformInteger, Range: [1, 3], Default: 3
learning_rate_init, Type: UniformFloat, Range: [1e-05, 1.0], Default: 0.001, on log-scale
width, Type: UniformInteger, Range: [16, 1024], Default: 64, on log-scale


/tmp/ipykernel_270325/2281861310.py:1: DeprecationWarning: Prefer using `list(space.values())` over `get_hyperparameters`
  search_space = "\n".join([str(hp) for hp in benchmark.get_configuration_space().get_hyperparameters()])


In [20]:
main_fidelity = benchmark.get_fidelity_space().get_hyperparameters()[0]
main_fidelity

/tmp/ipykernel_270325/3537652.py:1: DeprecationWarning: Prefer using `list(space.values())` over `get_hyperparameters`
  main_fidelity = benchmark.get_fidelity_space().get_hyperparameters()[0]


iter, Type: UniformInteger, Range: [3, 243], Default: 243

In [21]:
budget = 10
gt = GhosttunerSampler(model_type="sklearn MLPClassifier", search_space=search_space, use_cot=False, budget=budget)
random = RandomSampler()
tpe = TPESampler()
cmaes = CmaEsSampler()

In [22]:
study_rand = run_benchmark(random, benchmark, main_fidelity=main_fidelity, n_trials=budget)

[I 2023-12-13 21:39:31,530] A new study created in memory with name: no-name-e67896ef-e7d7-491a-8df9-e773e7a3b115
/tmp/ipykernel_270325/3060598696.py:5: DeprecationWarning: Prefer `space[name]` over `get_hyperparameter`
  hp = cs.get_hyperparameter(hp_name)


B


[I 2023-12-13 21:39:31,985] Trial 0 finished with value: 0.01045856798069189 and parameters: {'alpha': 6.111193287085821e-08, 'batch_size': 91, 'depth': 1, 'learning_rate_init': 0.013428642651629503, 'width': 47}. Best is trial 0 with value: 0.01045856798069189.
/tmp/ipykernel_270325/3060598696.py:5: DeprecationWarning: Prefer `space[name]` over `get_hyperparameter`
  hp = cs.get_hyperparameter(hp_name)
[I 2023-12-13 21:39:34,688] Trial 1 finished with value: 0.018503620273531807 and parameters: {'alpha': 7.640383656401834e-07, 'batch_size': 19, 'depth': 1, 'learning_rate_init': 0.0372549751830248, 'width': 16}. Best is trial 0 with value: 0.01045856798069189.
/tmp/ipykernel_270325/3060598696.py:5: DeprecationWarning: Prefer `space[name]` over `get_hyperparameter`
  hp = cs.get_hyperparameter(hp_name)
[I 2023-12-13 21:39:37,405] Trial 2 finished with value: 0.05953338696701527 and parameters: {'alpha': 3.6700939974789526e-08, 'batch_size': 10, 'depth': 2, 'learning_rate_init': 0.017773

In [23]:
study_gt = run_benchmark(gt, benchmark, main_fidelity=main_fidelity, n_trials=budget)

[I 2023-12-13 22:03:43,019] A new study created in memory with name: no-name-9c467122-250f-4910-b969-a4bb60e44226
/tmp/ipykernel_270325/3060598696.py:5: DeprecationWarning: Prefer `space[name]` over `get_hyperparameter`
  hp = cs.get_hyperparameter(hp_name)


B
ChatCompletion(id='chatcmpl-8VZ6t7Vk8C0lxRLf14nPoKmcsGvR3', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='{"alpha": 0.0001, "batch_size": 64, "depth": 2, "learning_rate_init": 0.01, "width": 128}', role='assistant', function_call=None, tool_calls=None))], created=1702533823, model='gpt-4-1106-preview', object='chat.completion', system_fingerprint='fp_6aca3b5ce1', usage=CompletionUsage(completion_tokens=38, prompt_tokens=282, total_tokens=320))


[I 2023-12-13 22:03:49,831] Trial 0 finished with value: 0.007240547063555924 and parameters: {'alpha': 0.0001, 'batch_size': 64, 'depth': 2, 'learning_rate_init': 0.01, 'width': 128}. Best is trial 0 with value: 0.007240547063555924.
/tmp/ipykernel_270325/3060598696.py:5: DeprecationWarning: Prefer `space[name]` over `get_hyperparameter`
  hp = cs.get_hyperparameter(hp_name)


ChatCompletion(id='chatcmpl-8VZ708HQyB08H8BsM6j6PQJjw2uu4', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='{"alpha": 0.00001, "batch_size": 128, "depth": 3, "learning_rate_init": 0.001, "width": 256}', role='assistant', function_call=None, tool_calls=None))], created=1702533830, model='gpt-4-1106-preview', object='chat.completion', system_fingerprint='fp_6aca3b5ce1', usage=CompletionUsage(completion_tokens=38, prompt_tokens=355, total_tokens=393))


[I 2023-12-13 22:03:57,054] Trial 1 finished with value: 0.008849557522123908 and parameters: {'alpha': 1e-05, 'batch_size': 128, 'depth': 3, 'learning_rate_init': 0.001, 'width': 256}. Best is trial 0 with value: 0.007240547063555924.
/tmp/ipykernel_270325/3060598696.py:5: DeprecationWarning: Prefer `space[name]` over `get_hyperparameter`
  hp = cs.get_hyperparameter(hp_name)


...

[I 2023-12-13 22:05:36,216] Trial 7 finished with value: 0.0032180209171359664 and parameters: {'alpha': 0.05, 'batch_size': 64, 'depth': 3, 'learning_rate_init': 0.0005, 'width': 128}. Best is trial 7 with value: 0.0032180209171359664.
/tmp/ipykernel_270325/3060598696.py:5: DeprecationWarning: Prefer `space[name]` over `get_hyperparameter`
  hp = cs.get_hyperparameter(hp_name)


ChatCompletion(id='chatcmpl-8VZ8iggaH9DJAq0A9PKfs90Zb35I0', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='{"alpha": 0.01, "batch_size": 32, "depth": 3, "learning_rate_init": 0.0001, "width": 256}', role='assistant', function_call=None, tool_calls=None))], created=1702533936, model='gpt-4-1106-preview', object='chat.completion', system_fingerprint='fp_b77c6a5c40', usage=CompletionUsage(completion_tokens=38, prompt_tokens=865, total_tokens=903))


[I 2023-12-13 22:06:28,812] Trial 8 finished with value: 0.006436041834271933 and parameters: {'alpha': 0.01, 'batch_size': 32, 'depth': 3, 'learning_rate_init': 0.0001, 'width': 256}. Best is trial 7 with value: 0.0032180209171359664.
/tmp/ipykernel_270325/3060598696.py:5: DeprecationWarning: Prefer `space[name]` over `get_hyperparameter`
  hp = cs.get_hyperparameter(hp_name)


ChatCompletion(id='chatcmpl-8VZ9ZJ7dEC9wXiQqFt56RZIaFHXaK', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='{"alpha": 0.005, "batch_size": 100, "depth": 2, "learning_rate_init": 0.0002, "width": 200}', role='assistant', function_call=None, tool_calls=None))], created=1702533989, model='gpt-4-1106-preview', object='chat.completion', system_fingerprint='fp_6aca3b5ce1', usage=CompletionUsage(completion_tokens=38, prompt_tokens=938, total_tokens=976))


[I 2023-12-13 22:06:37,488] Trial 9 finished with value: 0.00482703137570395 and parameters: {'alpha': 0.005, 'batch_size': 100, 'depth': 2, 'learning_rate_init': 0.0002, 'width': 200}. Best is trial 7 with value: 0.0032180209171359664.


In [24]:
study_tpe = run_benchmark(tpe, benchmark, main_fidelity=main_fidelity, n_trials=budget)

[I 2023-12-13 22:06:37,495] A new study created in memory with name: no-name-e3bfa8ad-e40f-4e0f-9070-d3d5a4591cf6
/tmp/ipykernel_270325/3060598696.py:5: DeprecationWarning: Prefer `space[name]` over `get_hyperparameter`
  hp = cs.get_hyperparameter(hp_name)


B


[I 2023-12-13 22:06:51,490] Trial 0 finished with value: 0.017699115044247815 and parameters: {'alpha': 0.2561379837724636, 'batch_size': 30, 'depth': 2, 'learning_rate_init': 0.0005458816192988445, 'width': 101}. Best is trial 0 with value: 0.017699115044247815.
/tmp/ipykernel_270325/3060598696.py:5: DeprecationWarning: Prefer `space[name]` over `get_hyperparameter`
  hp = cs.get_hyperparameter(hp_name)
[I 2023-12-13 22:07:17,153] Trial 1 finished with value: 0.006436041834271933 and parameters: {'alpha': 0.013191789618678817, 'batch_size': 32, 'depth': 3, 'learning_rate_init': 7.03358086635186e-05, 'width': 110}. Best is trial 1 with value: 0.006436041834271933.
/tmp/ipykernel_270325/3060598696.py:5: DeprecationWarning: Prefer `space[name]` over `get_hyperparameter`
  hp = cs.get_hyperparameter(hp_name)
[I 2023-12-13 22:07:27,216] Trial 2 finished with value: 0.01045856798069189 and parameters: {'alpha': 0.02965304508046866, 'batch_size': 26, 'depth': 3, 'learning_rate_init': 0.00020